A more up to date version is in 230818_allen_LFP_NWB-Zarr_plotBokeh.ipynb

TODO:
- [ ] add switch for overwriting zarr dir

In [ ]:
# input_filepath = os.path.join(data_dir, f"probe_{probe_id}_lfp.nwb")

# with h5py.File(input_filepath, "r") as f:
#     electrodes_table = f["general/extracellular_ephys/electrodes"]
#     electrode_metadata = {col_name: electrodes_table[col_name][:] for col_name in electrodes_table.keys()}


# import pandas as pd
# channel_multiindex = pd.MultiIndex.from_frame(pd.DataFrame(electrode_metadata), names=electrode_metadata.keys())

# pd.DataFrame(electrode_metadata)

In [ ]:
# def create_zarr_file_approach3(probe_id: str, data_dir: str, compress_data: bool = False, chunk_size: tuple = (1_000_000, 20), force_rechunk: bool = True):
#     """
#     Create a Zarr directory containing a single probe's data, chunked and optionally compressed.

#     Parameters
#     ----------
#     probe_id : str
#         The probe ID used to identify the specific probe.
#     data_dir : str
#         The base data directory where the input and output files are located.
#     compress_data : bool, optional
#         Whether to compress the LFP data. Compression reduces file size at the cost of additional computation.
#         Defaults to False.
#     chunk_size : tuple, optional
#         The size of chunks for the LFP data. Defaults to (1_000_000, 20) : (time_samples, channels).
#     force_rechunk : bool, optional
#         Whether to force rechunking with the provided chunk size, even if the HDF5 dataset is already chunked.
#         Defaults to True.

#     Returns
#     -------
#     ds : xr.Dataset
#         An Xarray Dataset object containing the LFP data and associated coordinates. The actual data is not loaded into
#         memory until explicitly accessed.
#     """

#     input_filepath = os.path.join(data_dir, f"probe_{probe_id}_lfp.nwb")
#     output_filepath = os.path.join(data_dir, f"lfp_{probe_id}.zarr")

#     with h5py.File(input_filepath, "r") as f:
#         lfp_dataset = f[f"acquisition/probe_{probe_id}_lfp/probe_{probe_id}_lfp_data/data"]
#         time_dataset = f[f"acquisition/probe_{probe_id}_lfp/probe_{probe_id}_lfp_data/timestamps"]
#         unit_attributes = {
#             "time_unit": time_dataset.attrs.get("unit", None),
#             "lfp_unit": lfp_dataset.attrs.get("unit", None),
#         }
#         reference_time = f.attrs.get("timestamps_reference_time", None)
#         session_description = f.attrs.get("session_description", None)
#         subject_info = {
#             "species": f["general/subject"].attrs.get("species", None),
#             "genotype": f["general/subject"].attrs.get("genotype", None),
#             "sex": f["general/subject"].attrs.get("sex", None),
#         }
#         electrodes_table = f["general/extracellular_ephys/electrodes"]
#         electrode_metadata = {col_name: electrodes_table[col_name][:] for col_name in electrodes_table.keys()}
        
#         # Creating MultiIndex for channel coordinate
#         channel_multiindex = pd.MultiIndex.from_frame(pd.DataFrame(electrode_metadata), names=electrode_metadata.keys())
#         channel_multiindex_coord = xr.DataArray(channel, name="channel", dims=["channel"], coords={"channel": channel_multiindex})
        
#         # if force_rechunk, use chunk_size.
#         # Else, if the hdf5 has chunk info, use that; if not, use chunk_size.
#         chunk_shape = chunk_size if force_rechunk else (lfp_dataset.chunks or chunk_size)
#         print(f'Chunking with {chunk_shape}')
        
#         # Read data in chunks and concat into a Dask array (first across channels then time)
#         arrays = []
#         for start_time in range(0, lfp_dataset.shape[0], chunk_shape[0]):
#             time_slice = slice(start_time, start_time + chunk_shape[0])
#             arrays_time = [da.from_array(lfp_dataset[time_slice, slice(start_channel, start_channel + chunk_shape[1])], chunks=chunk_shape)
#                            for start_channel in range(0, lfp_dataset.shape[1], chunk_shape[1])]
#             arrays.append(da.concatenate(arrays_time, axis=1))
#         lfp_dask_array = da.concatenate(arrays, axis=0)
#         time = time_dataset[:] # load in all the time, shouldn't be too big
#         # channel = np.arange(lfp_dataset.shape[1], dtype=np.uint32)

#     ds = xr.Dataset(
#         {
#             "lfp": (["time", "channel"], lfp_dask_array),
#         },
#         coords={
#             "time": time,
#             "channel": channel_multiindex_coord,
#         },
#         attrs={
#             "unit_attributes": unit_attributes,
#             "reference_time": reference_time,
#             "session_description": session_description,
#             "subject_info": subject_info,
#             "probe_info": probe_info,
#         }
#     )

#     if os.path.isdir(output_filepath):
#         shutil.rmtree(output_filepath)

#     compressor = Blosc(cname='zstd', clevel=3, shuffle=Blosc.BITSHUFFLE) if compress_data else None
#     ds.to_zarr(output_filepath, encoding={'lfp': {'compressor': compressor}})

#     print(f"Zarr file for probe {probe_id} has been created successfully at {output_filepath}.")

#     return ds
